In [64]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Multithreaded Architectures

Here is a "multithreaded" program.


In [47]:
compare([do_render_code("mv/matvec.c"), do_render_code("mv/matvec_threaded.c")])

## Simultaneous Multithreading (SMT)

Let's take a look how the threaded program would work on an SMT core -- we use `taskset` to fix both threads on the same physical core

In [49]:
! make -C  mv clean all
! echo ",IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! echo -n "single_thread," >> stats.csv
! taskset -c 11 ./mv/matvec 16384
! echo -n "two_threads_smt," >> stats.csv
! taskset -c 10,11 ./mv/matvec_threaded 16384
display_df_mono(render_csv("stats.csv"))
!cp stats.csv matvec_SMT.csv

make: Entering directory '/nfshome/htseng/courses/CSE142/demo/multiprocessor/mv'
rm -f matvec matvec_threaded
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 -pthread  matvec.c perfstats.c -o matvec
matvec.c: In function ‘main’:
matvec.c:53:22: warning: zero-length gnu_printf format string []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wformat-zero-length-Wformat-zero-length]8;;]
   53 |     sprintf(preamble,"");
      |                      ^~
perfstats.c: In function ‘change_cpufrequnecy’:
perfstats.c:115:11: warning: implicit declaration of function ‘sched_getcpu’; did you mean ‘SYS_getcpu’? []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wimplicit-function-declaration-Wimplicit-function-declaration]8;;]
  115 |     cpu = sched_getcpu();
      |           ^~~~~~~~~~~~
      |           SYS_getcpu
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3 -pthread  matvec_threaded.c perfstats.c -o matvec_threaded
matvec_threaded.c: In function ‘main’:
matvec_threaded.c:82

,index,Unnamed: 0,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,single_thread,1209488580,633890018,0.524098,0.192880,0.122265,0.018799,5056337,268967316,134500818,33583
1,1,two_threads_smt,1209879840,802687955,0.663444,0.096764,0.077671,0.035983,9683012,269103198,134567817,35516


In [50]:
! echo "IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! taskset -c 10,11 ./mv/matvec 16384 & taskset -c 10,11 ./mv/matvec 16384
display_df_mono(render_csv("stats.csv"))
!cp stats.csv two_identical_smt.csv

41845914.000000,41745652
41845914.000000,41745652


,index,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,1209776394,791646272,0.654374,0.192794,0.152625,0.041119,11064072,269076299,134549633,33207
1,1,1209670659,796024847,0.658051,0.192823,0.153492,0.041076,11050104,269019213,134536805,33787


## Chip Multiprocessors (CMP)

Let's try run them on separate cores.

In [60]:
! echo "IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! taskset -c 10 ./mv/matvec 16384 & taskset -c 0 ./mv/matvec 16384
display_df_mono(render_csv("stats.csv"))
!cp stats.csv two_identical_cmp.csv

41845914.000000,41745652
41845914.000000,41745652


,index,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,1209103637,678782202,0.561393,0.192817,0.130881,0.033145,8910420,268833259,134431949,33852
1,1,1209052887,665116396,0.550114,0.196469,0.130675,0.019520,5247815,268840367,134415552,26709


In [ ]:
## SMT vs CMP

In [63]:
! echo "IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses" > stats.csv
! taskset -c 10,11 ./mv/matvec 16384 & taskset -c 10,11 ../OoO_programming/popcounts/popcount_D
display_df_mono(render_csv("stats.csv"))
!cp stats.csv two_different_smt.csv

41845914.000000,41745652
Result: 32000010811


,index,IC,Cycles,CPI,CT,ET,L1_dcache_miss_rate,L1_dcache_misses,L1_dcache_accesses,branches,branch_misses
0,0,1209531616,675056645,0.558114,0.192825,0.130168,0.021720,5842479,268988239,134506720,28691
